---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
import re 
from datetime import datetime
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    #How to Add a New Line????
    file = open('university_towns.txt', "r")
    lines = file.read().splitlines()
    file.close()
    
    RegionName = []
    State = []
    df = pd.DataFrame(columns = ['State','RegionName'])    

    for i in lines:
        if '[edit]' in i:
            Interim = i.replace('[edit]',"")
            # Interim stores the state name in the local space
            continue
        
        if '(' in i: 
            ab = re.sub(r' \(.*?\)',"",i)
            RegionName.append(re.sub(r'\[.*?\]',"",ab))
            # Only when we very the line is a county, do we append our State list using 'Interim'
            State.append(Interim.strip())

    df['State'] = State
    df['RegionName'] = RegionName
    return df

get_list_of_university_towns()


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
508,Wisconsin,River Falls
509,Wisconsin,Stevens Point
510,Wisconsin,Waukesha
511,Wisconsin,Whitewater


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdplev = pd.read_excel('gdplev.xls',skiprows = range(1,218))
    df = gdplev[['Unnamed: 4','Unnamed: 5','Unnamed: 6']]
    df.rename(columns = {'Unnamed: 4': 'Date','Unnamed: 5':'GDP_2009','Unnamed: 6':'GDP'}, inplace = True)

    i = 2
    while i < df.shape[0]:
        if df['GDP_2009'][i-1] < df['GDP_2009'][i-2] and df['GDP_2009'][i] < df['GDP_2009'][i-1]:
            i -= 2
            return df['Date'][i]
            break
        else:
            i += 1
                 
    # Not required, but can just write: return df['Date'][i]

get_recession_start()

C:\Users\Marshall Too\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
        
    gdplev = pd.read_excel('gdplev.xls',skiprows = range(1,218))
    df = gdplev[['Unnamed: 4','Unnamed: 5','Unnamed: 6']]
    df.rename(columns = {'Unnamed: 4': 'Date','Unnamed: 5':'GDP_2009','Unnamed: 6':'GDP'}, inplace = True)

    # Compute Recession Start to Index 
    Recession_start = get_recession_start()
    i = df[df['Date'] == Recession_start].index.values[0]
    # Start with the next period (Why Is this wrong??)
    i += 1  
    
    while i < df.shape[0]:
        if df['GDP_2009'][i] > df['GDP_2009'][i-1] and df['GDP_2009'][i-1] > df['GDP_2009'][i-2]:
            return df['Date'][i]
            break
        else:
            i += 1
    return Date
            
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gdplev = pd.read_excel('gdplev.xls',skiprows = range(1,218))
    df = gdplev[['Unnamed: 4','Unnamed: 5','Unnamed: 6']]
    df.rename(columns = {'Unnamed: 4': 'Date','Unnamed: 5':'GDP_2009','Unnamed: 6':'GDP'}, inplace = True)

    start = get_recession_start()
    end = get_recession_end()
    start_no = df[df['Date']== start].index.values[0]
    end_no = df[df['Date']== end].index.values[0]
    
    #filter DF
    df = df[start_no:end_no]
    a = df.loc[df['GDP'].idxmin()]['Date']
    return a

get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    
    A quarter is a specific three month period, Q1 is January through March, Q2 is April through June, 
    Q3 is July through September, Q4 is October through December.
    
    '''
    hdata = pd.read_csv('City_Zhvi_AllHomes.csv')
    hdata = hdata.transpose()
    hdata = hdata.reset_index()
    
    # Extract out Month to Quart in separate df
    quart = hdata.iloc[6:] 
    
    # Convert to Datetime Format (nts, drop only works with index)
    quart['index'] = pd.to_datetime(quart['index'], format='%Y-%m')
    quart = quart.drop(quart[quart['index'].dt.year < 2000].index, axis=0)
    quart = quart.set_index('index')
    #Another way to sum: df1 = pd.pivot_table(quart, index= quart['index'],values=['Define Colum Names here'],aggfunc=np.sum)
    df1 = quart.apply(pd.to_numeric, errors='coerce')
    df1 = df1.resample('Q').mean()
    df1 = df1.reset_index()
    
    #Change Datetime format to Quarterly, PeriodIndex changes Start/End Dates to Quarterly
    df1['index'] = pd.PeriodIndex(pd.to_datetime(df1['index']), freq='q')

    # Define 2nd to combine
    df2 = hdata.iloc[0:6]
    
    #2 Approach; M1, add every 3 columns - M2, reshape and sum by date
    df3 = pd.concat([df2,df1])
    df3 = df3.set_index('index')
    
    #3 Transpose Df correctly
    df3 = df3.transpose()
    df3['State'] = df3['State'].map(states)
    df3 = df3.set_index(['State','RegionName'])
    df3 = df3.drop(['RegionID','Metro','CountyName','SizeRank'], axis=1)
    return df3

convert_housing_data_to_quarters()

C:\Users\Marshall Too\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515467,522800,528067,532267,540800,557200,572833,582867,591633,587200
California,Los Angeles,207067,214467,220967,226167,233000,239100,245067,253033,261967,272700,...,498033,509067,518867,528800,538167,547267,557733,566033,577467,584050
Illinois,Chicago,138400,143633,147867,152133,156933,161800,166400,170433,175500,177567,...,192633,195767,201267,201067,206033,208300,207900,206067,208200,212000
Pennsylvania,Philadelphia,53000,53633.3,54133.3,54700,55333.3,55533.3,56266.7,57533.3,59133.3,60733.3,...,113733,115300,115667,116200,117967,121233,122200,123433,126933,128700
Arizona,Phoenix,111833,114367,116000,117400,119600,121567,122700,124300,126533,128367,...,164267,165367,168500,171533,174167,179067,183833,187900,191433,195200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101767,105400,111367,114867,125967,129900,129900,129433,131900,134200,...,144867,146867,149233,148667,149333,149867,149933,149833,151267,155000
New York,Urbana,79200,81666.7,91700,98366.7,94866.7,98533.3,102967,98033.3,93966.7,94600,...,132133,137033,140067,141700,137867,136467,136167,138967,144200,143000
Wisconsin,New Denmark,114567,119267,126067,131967,143800,146967,148367,149167,153133,156733,...,174567,181167,186167,187600,188667,188433,188933,191067,192833,197600


In [46]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    unitowns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    hdata = convert_housing_data_to_quarters()    

    #Convert hdata columns to string so they can match bottom/start
    ab = hdata.columns.astype(str)
    ab = ab.str.lower()
    hdata.columns = ab

    #Define before start
    bstart = hdata.columns[hdata.columns.get_loc(start) -1]

    #Compute Ratio
    hdata['ratio'] = hdata[bstart] - hdata[bottom]
    df4 = hdata[['ratio',bstart,bottom]]
    # hdata = hdata[[bottom,bstart,'ratio']]
    # hdata = hdata.reset_index()

    #Join hdata with unitowns columns
    unitowns['True'] = True
    df6 = unitowns
    df6 = df6.set_index(['State','RegionName'])
    # Note df4 = 10k regions, Note Unitowns = 500, Therefore left join Unitowns to merge

    df5 = df4.merge(df6, left_index = True, right_index = True, how ='left')
    df5['True'] = df5['True'].fillna(False)

    unitowns = df5[df5['True']==True] 
    non_unitowns =df5[df5['True']==False]
    #Runttest: Rmb to remove NA values
    st, d = ttest_ind(unitowns['ratio'].dropna(), non_unitowns['ratio'].dropna(), equal_var=True)

    if d < 0.05:
        different = True
    else:
        different = False

    if unitowns['ratio'].mean() < non_unitowns['ratio'].mean():
        better='university town'
    else:
        better='non-university town'

    return (different,d, better)

run_ttest()

C:\Users\Marshall Too\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(True, 0.0008103477542188203, 'university town')